In [1]:
import tkinter as tk
from tkinter import filedialog as fd
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
import os

# import import_ipynb
# %run LESK32.ipynb

In [62]:
path = r'C:\Users\lopezb41\OneDrive - imec\Documents\Experiments\Data\Electrochemical\L22\L22_TiO2_90\1C\lith'
files = [os.path.join(path, i) for i in os.listdir(path)]
new_files = []
for i in files:
    directory, filename = os.path.split(i)
    numeric_part = filename.split('_')[-1].split('.txt')[0]
    numeric_two_decimals = '{:02d}'.format(int(numeric_part))
    
    new_filename =filename[:-7] + filename[-7:-4].replace(numeric_part, numeric_two_decimals) + '.txt'  # filename.replace(numeric_part, numeric_two_decimals)
    new_file_path = os.path.join(directory, new_filename)
    
    if i != new_file_path:
                
        os.rename(i, new_file_path)
        print(f"Renamed: {i} => {new_file_path}")

    

['C:\\Users\\lopezb41\\OneDrive - imec\\Documents\\Experiments\\Data\\Electrochemical\\L22\\L22_TiO2_90\\1C\\lith\\L22_TiO2_90_1C_lith_1C_01.txt',
 'C:\\Users\\lopezb41\\OneDrive - imec\\Documents\\Experiments\\Data\\Electrochemical\\L22\\L22_TiO2_90\\1C\\lith\\L22_TiO2_90_1C_lith_1C_02.txt',
 'C:\\Users\\lopezb41\\OneDrive - imec\\Documents\\Experiments\\Data\\Electrochemical\\L22\\L22_TiO2_90\\1C\\lith\\L22_TiO2_90_1C_lith_1C_03.txt',
 'C:\\Users\\lopezb41\\OneDrive - imec\\Documents\\Experiments\\Data\\Electrochemical\\L22\\L22_TiO2_90\\1C\\lith\\L22_TiO2_90_1C_lith_1C_04.txt',
 'C:\\Users\\lopezb41\\OneDrive - imec\\Documents\\Experiments\\Data\\Electrochemical\\L22\\L22_TiO2_90\\1C\\lith\\L22_TiO2_90_1C_lith_1C_05.txt',
 'C:\\Users\\lopezb41\\OneDrive - imec\\Documents\\Experiments\\Data\\Electrochemical\\L22\\L22_TiO2_90\\1C\\lith\\L22_TiO2_90_1C_lith_1C_06.txt',
 'C:\\Users\\lopezb41\\OneDrive - imec\\Documents\\Experiments\\Data\\Electrochemical\\L22\\L22_TiO2_90\\1C\\lith\\L22

In [8]:
#warburg
zw = 400.75
tau = 18.303
alpha = 0.69

print(34 * 9e-5)
print((16+7) * 0.035)

0.0030600000000000002
0.805


In [4]:
diameter = 1.5 # cm
radius = diameter / 2
area = np.pi * radius**2
print(0.05/area)
area

0.02829421210522584


1.7671458676442586

In [2]:
r = 1.5/2
area = np.pi * r**2
area

1.7671458676442586

In [3]:
a = 200 / 0.785
b = 10 / 3.1416
print(a,b)

254.77707006369425 3.183091418385536


In [4]:
areal_density = 90e15 # at/cm2 of Li
thickness_th = 68e-7 # cm ; 68 nm
li_density = areal_density * thickness_th
avo = 6.022e23 # 1/mol
li_mol = li_density / avo
print(li_mol * 1000)
print(100 * 100)

1.0162736632348058e-09
10000
